In [ ]:
# pip install librosa
# pip install keras
# pip install tensorflow
# pip install os

In [10]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical


import module.lstm as LSTM_MODEL
import module.preprocess as PROCESSING

In [3]:
encoder = LabelEncoder()
SR = 16000

ROOT = "C:/Users/SSAFY/Desktop/특화프로젝트"
TRAIN_PATH = f"{ROOT}/train"
VAL_PATH = f"{ROOT}/val"
TENSER_PATH = f"{ROOT}/tensor"
CHECKPOINT_PATH = f"{ROOT}/checkpoint"

In [4]:
# train_mfccs, train_labels = PROCESSING.createDATA(TRAIN_PATH,SR)
train_mfccs, train_labels = np.load(f'{TENSER_PATH}/train_mfccs.npy'), np.load(f'{TENSER_PATH}/train_labels.npy')

In [5]:
#val_mfccs, val_labels = PROCESSING.createDATA(VAL_PATH,SR)
val_mfccs, val_labels = np.load(f'{TENSER_PATH}/val_mfccs.npy'), np.load(f'{TENSER_PATH}/val_labels.npy')

In [6]:
# np.save(f'{TENSER_PATH}/train_mfccs.npy', train_mfccs)
# np.save(f'{TENSER_PATH}/train_labels.npy', train_labels)
# np.save(f'{TENSER_PATH}/val_mfccs.npy', val_mfccs)
# np.save(f'{TENSER_PATH}/val_labels.npy', val_labels)

In [7]:
encoder.fit(np.concatenate([train_labels, val_labels]))

train_y = encoder.transform(train_labels)
val_y = encoder.transform(val_labels)

In [8]:
CHECK_POINT_FILE = "only_mfcc/checkpoint.h5"
model = LSTM_MODEL.create_lstm_model(len(encoder.classes_),sr = SR)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
checkpoint_callback = LSTM_MODEL.Checkpoint(f"{CHECKPOINT_PATH}/{CHECK_POINT_FILE}")


if os.path.isfile(f"{CHECKPOINT_PATH}/{CHECK_POINT_FILE}"):
    model.load_weights(f"{CHECKPOINT_PATH}/{CHECK_POINT_FILE}")


In [ ]:
model.fit(train_mfccs, to_categorical(train_y,num_classes=len(encoder.classes_)), epochs=50, validation_data=(val_mfccs, to_categorical(val_y,num_classes=len(encoder.classes_))), callbacks=[checkpoint_callback], initial_epoch=0)
#model.save(f"{CHECKPOINT_PATH}/{CHECK_POINT_FILE}")

In [13]:
test_wav = PROCESSING.load_wav_file(f"{ROOT}/샘플데이터/윤하.wav")
test_wav = PROCESSING.preprocess_mfccs([test_wav])

In [14]:
pred = model.predict(test_wav)

1/1 [==============================] - 0s 487ms/step


In [15]:
pred = model.predict(test_wav)
top_5 = np.argsort(pred, axis=1)[:, -5:]
top_5_labels = encoder.inverse_transform(top_5.T)

1/1 [==============================] - 0s 162ms/step


c:\Users\SSAFY\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
top_5_labels

array(['SINGER_52', 'SINGER_51', 'SINGER_36', 'SINGER_43', 'SINGER_44'],
      dtype='<U9')

In [ ]:
pred